# Performance Blow-out

## Description
When agents maximize their time when working on these activities by stretching their work, acting as if they are doing the work, or by falsely recording end times.

In [1]:
import pandas as pd
import numpy as np
import altair as alt

In [2]:
log_name = 'propr_test'

fn_logs = {
    'propr': '../data/preproc/proprietary.csv',
    'propr_ref': '../data/preproc/propr_train-70.csv',
    'propr_test': '../data/preproc/propr_test-30.csv'
}

In [3]:
log = pd.read_csv(fn_logs[log_name])
log

,activity,case_id,start timestamp,complete timestamp,resource,Weekday,Cost,"School holidays; 0 = no, 1 = yes",Approval; 1 = low risk to 4 = high risk,Type of loan; 0 = normal; 1 = special,_start timestamp,_complete timestamp
0,Precheck,ID-2,6/05/2011 16:09,6/05/2011 16:27,090-10-02,Friday,"0,87 € per minute",0.0,1.0,0.0,2011-05-06 16:09:00,2011-05-06 16:27:00
1,Application_Processing_Branches,ID-2,9/05/2011 11:14,9/05/2011 11:16,010-23-13,Monday,"1,02 € per minute",0.0,1.0,0.0,2011-05-09 11:14:00,2011-05-09 11:16:00
2,Precheck,ID-2,9/05/2011 15:44,9/05/2011 15:58,090-10-02,Monday,"0,87 € per minute",0.0,1.0,0.0,2011-05-09 15:44:00,2011-05-09 15:58:00
3,Processing_Incomplete_Orders,ID-2,9/05/2011 16:10,9/05/2011 16:10,010-23-13,Monday,"1,02 € per minute",0.0,1.0,0.0,2011-05-09 16:10:00,2011-05-09 16:10:00
4,Application_Processing_Branches,ID-2,10/05/2011 8:52,10/05/2011 9:32,010-23-13,Tuesday,"1,02 € per minute",0.0,1.0,0.0,2011-05-10 08:52:00,2011-05-10 09:32:00
...,...,...,...,...,...,...,...,...,...,...,...,...
5285,Precheck,ID-1720,20/05/2011 17:24,20/05/2011 17:26,000-3-01,Friday,"0,87 € per minute",0.0,2.0,0.0,2011-05-20 17:24:00,2011-05-20 17:26:00
5286,Check_of_Processing_Applications,ID-1720,23/05/2011 15:53,23/05/2011 16:06,000-2-01,Monday,"0,87 € per minute",0.0,2.0,0.0,2011-05-23 15:53:00,2011-05-23 16:06:00
5287,Processing_of_Applications,ID-1720,26/05/2011 9:25,26/05/2011 9:25,010-23-07,Thursday,"1,02 € per minute",0.0,2.0,0.0,2011-05-26 09:25:00,2011-05-26 09:25:00
5288,Archieving,ID-1720,26/05/2011 10:38,26/05/2011 10:47,010-23-07,Thursday,"1,02 € per minute",0.0,2.0,0.0,2011-05-26 10:38:00,2011-05-26 10:47:00


In [4]:
# add activity duration
log['complete timestamp'] = pd.to_datetime(log['complete timestamp'], dayfirst=True)
log['start timestamp'] = pd.to_datetime(log['start timestamp'], dayfirst=True)
log['activity_duration_minute'] = (
    log['complete timestamp'] -
    log['start timestamp']
).dt.total_seconds() / 60
log

,activity,case_id,start timestamp,complete timestamp,resource,Weekday,Cost,"School holidays; 0 = no, 1 = yes",Approval; 1 = low risk to 4 = high risk,Type of loan; 0 = normal; 1 = special,_start timestamp,_complete timestamp,activity_duration_minute
0,Precheck,ID-2,2011-05-06 16:09:00,2011-05-06 16:27:00,090-10-02,Friday,"0,87 € per minute",0.0,1.0,0.0,2011-05-06 16:09:00,2011-05-06 16:27:00,18.0
1,Application_Processing_Branches,ID-2,2011-05-09 11:14:00,2011-05-09 11:16:00,010-23-13,Monday,"1,02 € per minute",0.0,1.0,0.0,2011-05-09 11:14:00,2011-05-09 11:16:00,2.0
2,Precheck,ID-2,2011-05-09 15:44:00,2011-05-09 15:58:00,090-10-02,Monday,"0,87 € per minute",0.0,1.0,0.0,2011-05-09 15:44:00,2011-05-09 15:58:00,14.0
3,Processing_Incomplete_Orders,ID-2,2011-05-09 16:10:00,2011-05-09 16:10:00,010-23-13,Monday,"1,02 € per minute",0.0,1.0,0.0,2011-05-09 16:10:00,2011-05-09 16:10:00,0.0
4,Application_Processing_Branches,ID-2,2011-05-10 08:52:00,2011-05-10 09:32:00,010-23-13,Tuesday,"1,02 € per minute",0.0,1.0,0.0,2011-05-10 08:52:00,2011-05-10 09:32:00,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5285,Precheck,ID-1720,2011-05-20 17:24:00,2011-05-20 17:26:00,000-3-01,Friday,"0,87 € per minute",0.0,2.0,0.0,2011-05-20 17:24:00,2011-05-20 17:26:00,2.0
5286,Check_of_Processing_Applications,ID-1720,2011-05-23 15:53:00,2011-05-23 16:06:00,000-2-01,Monday,"0,87 € per minute",0.0,2.0,0.0,2011-05-23 15:53:00,2011-05-23 16:06:00,13.0
5287,Processing_of_Applications,ID-1720,2011-05-26 09:25:00,2011-05-26 09:25:00,010-23-07,Thursday,"1,02 € per minute",0.0,2.0,0.0,2011-05-26 09:25:00,2011-05-26 09:25:00,0.0
5288,Archieving,ID-1720,2011-05-26 10:38:00,2011-05-26 10:47:00,010-23-07,Thursday,"1,02 € per minute",0.0,2.0,0.0,2011-05-26 10:38:00,2011-05-26 10:47:00,9.0


In [5]:
# encode activity labels as short names for presentation purpose

print('There exist {} unique activity labels'.format(
    log['activity'].nunique()
))

# use initial letters for encoding
act_labels_short = dict()
for label in log['activity'].unique():
    label_short = ''.join([w[0] for w in label.split('_')])
    act_labels_short[label] = label_short
print('There are {} short labels after encoding'.format(
    len(act_labels_short.values())
))
for k in sorted(act_labels_short.keys()):
    print('{:>50} -> {:>30}'.format(k, act_labels_short[k]))

log['activity_short'] = log['activity'].apply(
    lambda x: act_labels_short[x]
)

log['activity'] = log['activity_short']

There exist 25 unique activity labels
There are 25 short labels after encoding
                              Administrative_Tasks ->                             AT
                   Application_Processing_Branches ->                            APB
                                          Approval ->                              A
                                   Approval_Branch ->                             AB
                          Approval_Executive_Board ->                            AEB
                                        Archieving ->                              A
                                 Check_of_Approval ->                            CoA
                                Check_of_Documents ->                            CoD
                  Check_of_Processing_Applications ->                           CoPA
                                               End ->                              E
                                   Further_inquiry ->                  

## Performance-PB-1

### Profile work by activity types and/or case types

In [6]:
# proprietary: 
# case types as types of loan
log['case_type'] = log['Type of loan; 0 = normal; 1 = special'].apply(
    lambda x: 'special loan' if x == 1 else 'normal loan'
)

case_types = list(log['case_type'].unique())
log

,activity,case_id,start timestamp,complete timestamp,resource,Weekday,Cost,"School holidays; 0 = no, 1 = yes",Approval; 1 = low risk to 4 = high risk,Type of loan; 0 = normal; 1 = special,_start timestamp,_complete timestamp,activity_duration_minute,activity_short,case_type
0,P,ID-2,2011-05-06 16:09:00,2011-05-06 16:27:00,090-10-02,Friday,"0,87 € per minute",0.0,1.0,0.0,2011-05-06 16:09:00,2011-05-06 16:27:00,18.0,P,normal loan
1,APB,ID-2,2011-05-09 11:14:00,2011-05-09 11:16:00,010-23-13,Monday,"1,02 € per minute",0.0,1.0,0.0,2011-05-09 11:14:00,2011-05-09 11:16:00,2.0,APB,normal loan
2,P,ID-2,2011-05-09 15:44:00,2011-05-09 15:58:00,090-10-02,Monday,"0,87 € per minute",0.0,1.0,0.0,2011-05-09 15:44:00,2011-05-09 15:58:00,14.0,P,normal loan
3,PIO,ID-2,2011-05-09 16:10:00,2011-05-09 16:10:00,010-23-13,Monday,"1,02 € per minute",0.0,1.0,0.0,2011-05-09 16:10:00,2011-05-09 16:10:00,0.0,PIO,normal loan
4,APB,ID-2,2011-05-10 08:52:00,2011-05-10 09:32:00,010-23-13,Tuesday,"1,02 € per minute",0.0,1.0,0.0,2011-05-10 08:52:00,2011-05-10 09:32:00,40.0,APB,normal loan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5285,P,ID-1720,2011-05-20 17:24:00,2011-05-20 17:26:00,000-3-01,Friday,"0,87 € per minute",0.0,2.0,0.0,2011-05-20 17:24:00,2011-05-20 17:26:00,2.0,P,normal loan
5286,CoPA,ID-1720,2011-05-23 15:53:00,2011-05-23 16:06:00,000-2-01,Monday,"0,87 € per minute",0.0,2.0,0.0,2011-05-23 15:53:00,2011-05-23 16:06:00,13.0,CoPA,normal loan
5287,PoA,ID-1720,2011-05-26 09:25:00,2011-05-26 09:25:00,010-23-07,Thursday,"1,02 € per minute",0.0,2.0,0.0,2011-05-26 09:25:00,2011-05-26 09:25:00,0.0,PoA,normal loan
5288,A,ID-1720,2011-05-26 10:38:00,2011-05-26 10:47:00,010-23-07,Thursday,"1,02 € per minute",0.0,2.0,0.0,2011-05-26 10:38:00,2011-05-26 10:47:00,9.0,A,normal loan


### Determine groups of comparable resources

In [7]:
# proprietary
log['department'] = log['resource'].apply(
    lambda x: x.split('-')[0]
)
log['department_role'] = log['resource'].apply(
    lambda x: x.split('-')[0] + '-' + x.split('-')[1]
)
log

,activity,case_id,start timestamp,complete timestamp,resource,Weekday,Cost,"School holidays; 0 = no, 1 = yes",Approval; 1 = low risk to 4 = high risk,Type of loan; 0 = normal; 1 = special,_start timestamp,_complete timestamp,activity_duration_minute,activity_short,case_type,department,department_role
0,P,ID-2,2011-05-06 16:09:00,2011-05-06 16:27:00,090-10-02,Friday,"0,87 € per minute",0.0,1.0,0.0,2011-05-06 16:09:00,2011-05-06 16:27:00,18.0,P,normal loan,090,090-10
1,APB,ID-2,2011-05-09 11:14:00,2011-05-09 11:16:00,010-23-13,Monday,"1,02 € per minute",0.0,1.0,0.0,2011-05-09 11:14:00,2011-05-09 11:16:00,2.0,APB,normal loan,010,010-23
2,P,ID-2,2011-05-09 15:44:00,2011-05-09 15:58:00,090-10-02,Monday,"0,87 € per minute",0.0,1.0,0.0,2011-05-09 15:44:00,2011-05-09 15:58:00,14.0,P,normal loan,090,090-10
3,PIO,ID-2,2011-05-09 16:10:00,2011-05-09 16:10:00,010-23-13,Monday,"1,02 € per minute",0.0,1.0,0.0,2011-05-09 16:10:00,2011-05-09 16:10:00,0.0,PIO,normal loan,010,010-23
4,APB,ID-2,2011-05-10 08:52:00,2011-05-10 09:32:00,010-23-13,Tuesday,"1,02 € per minute",0.0,1.0,0.0,2011-05-10 08:52:00,2011-05-10 09:32:00,40.0,APB,normal loan,010,010-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5285,P,ID-1720,2011-05-20 17:24:00,2011-05-20 17:26:00,000-3-01,Friday,"0,87 € per minute",0.0,2.0,0.0,2011-05-20 17:24:00,2011-05-20 17:26:00,2.0,P,normal loan,000,000-3
5286,CoPA,ID-1720,2011-05-23 15:53:00,2011-05-23 16:06:00,000-2-01,Monday,"0,87 € per minute",0.0,2.0,0.0,2011-05-23 15:53:00,2011-05-23 16:06:00,13.0,CoPA,normal loan,000,000-2
5287,PoA,ID-1720,2011-05-26 09:25:00,2011-05-26 09:25:00,010-23-07,Thursday,"1,02 € per minute",0.0,2.0,0.0,2011-05-26 09:25:00,2011-05-26 09:25:00,0.0,PoA,normal loan,010,010-23
5288,A,ID-1720,2011-05-26 10:38:00,2011-05-26 10:47:00,010-23-07,Thursday,"1,02 € per minute",0.0,2.0,0.0,2011-05-26 10:38:00,2011-05-26 10:47:00,9.0,A,normal loan,010,010-23


In [8]:
# proprietary
# select resources from the same department_role
sel_resources = set(log.loc[
    log['department_role'] == '010-24',
    'resource'
].unique())

### Profile group member performance by activity duration

In [9]:
# use average activity duration for the same activity
# add a column for normalized values for plotting
max_act_dur = log.groupby(
    ['activity', 'case_type']
).agg(
    max_activity_duration_minute=pd.NamedAgg(
        'activity_duration_minute', 'max'
    )
).reset_index()
res_performance = log.groupby(
    ['resource', 'activity', 'case_type']
).agg(
    mean_activity_duration_minute=pd.NamedAgg(
        'activity_duration_minute', 'mean'
    )
).reset_index().merge(
    max_act_dur, how='left', on=['activity', 'case_type']
)
res_performance['normalized_mean_activity_duration_minute'] = (
    res_performance['mean_activity_duration_minute'] / 
    res_performance['max_activity_duration_minute']
)
res_performance = res_performance[
    res_performance['resource'].isin(sel_resources)
]
res_performance

,resource,activity,case_type,mean_activity_duration_minute,max_activity_duration_minute,normalized_mean_activity_duration_minute
199,010-24-01,A,normal loan,13.372093,301.0,0.044426
200,010-24-01,A,special loan,12.666667,286.0,0.044289
201,010-24-01,CoPA,normal loan,0.500000,217.0,0.002304
202,010-24-01,CoPA,special loan,0.000000,30.0,0.000000
203,010-24-01,E,normal loan,0.000000,1.0,0.000000
...,...,...,...,...,...,...
303,010-24-26,PoAS,special loan,2.833333,65.0,0.043590
304,010-24-26,R,special loan,1.000000,1.0,1.000000
305,010-24-26,RB,normal loan,9.350000,269.0,0.034758
306,010-24-26,RB,special loan,18.000000,284.0,0.063380


### Identify resources who spent significantly longer time on the same type of work compared to others

In [10]:
data = res_performance[
    (res_performance['resource'].isin(sel_resources)) &
    (res_performance['case_type'] == 'special loan')
]
alt.layer(
    alt.Chart(data, title='mean activity duration (minutes)').mark_rect().encode(
        y=alt.Y('resource:N').title('resource ID'),
        x=alt.X('activity:N').title('activity'),
        color=alt.Color('normalized_mean_activity_duration_minute').scale(scheme='lightgreyred').legend(None),
        tooltip=['resource', 'activity', 'case_type', 'mean_activity_duration_minute', 'max_activity_duration_minute']
    ),
    alt.Chart(data).mark_text(baseline='middle').encode(
        y=alt.Y('resource:N').title('resource ID'),
        x=alt.X('activity:N').title('activity'),
        text=alt.Text('mean_activity_duration_minute', format='.0f')
    ),
)

alt.LayerChart(...)

In [11]:
log_data = log[
    (log['resource'].isin(sel_resources)) &
    (log['case_type'] == 'special loan')
]

res_data = res_performance[
    (res_performance['resource'].isin(sel_resources)) &
    (res_performance['case_type'] == 'special loan')
]

charts = []
act_dur_thresholds = []
for activity in res_data['activity'].unique():
    # Show only visualizations of certain activities
    threshold = log_data.loc[log_data['activity'] == activity, 'activity_duration_minute'].quantile(0.75) + 1.5 * (
        log_data.loc[log_data['activity'] == activity,'activity_duration_minute'].quantile(0.75) -
        log_data.loc[log_data['activity'] == activity,'activity_duration_minute'].quantile(0.25)
    )
    print('Activity "{}" mean duration time threshold (Q3 + 1.5IQR): {}'.format(
        activity, threshold
    ))
    act_dur_thresholds.append({
        'activity': activity,
        'threshold': threshold
    })
    subset = res_data[res_data['activity'] == activity]

    selection = alt.selection_point(fields=['resource'], bind='legend')
    charts.append(
        alt.layer(
            alt.Chart(log_data[log_data['activity'] == activity]).mark_boxplot(
                outliers={'size': 0}
            ).encode(
                y='activity',
                x=alt.X('activity_duration_minute').title(
                    'Activity duration time (minute)'
                ),
                color=alt.value('grey'),
                opacity=alt.value(0.5)
            ),
            alt.Chart(log_data[log_data['activity'] == activity]).mark_circle().encode(
                y='activity',
                x='activity_duration_minute',
                color='resource',
                tooltip=['resource', 'activity', 'activity_duration_minute'],
                opacity=alt.condition(selection, alt.value(1), alt.value(0))
            ).add_params(
                selection
            )
        )
    )
act_dur_thresholds = pd.DataFrame(act_dur_thresholds)
alt.vconcat(*charts)

Activity "A" mean duration time threshold (Q3 + 1.5IQR): 49.0
Activity "CoPA" mean duration time threshold (Q3 + 1.5IQR): 24.625
Activity "E" mean duration time threshold (Q3 + 1.5IQR): 0.0
Activity "GSoC" mean duration time threshold (Q3 + 1.5IQR): 330.625
Activity "P" mean duration time threshold (Q3 + 1.5IQR): 18.5
Activity "PoA" mean duration time threshold (Q3 + 1.5IQR): 75.0
Activity "RB" mean duration time threshold (Q3 + 1.5IQR): 43.75
Activity "SvK" mean duration time threshold (Q3 + 1.5IQR): 9.0
Activity "AT" mean duration time threshold (Q3 + 1.5IQR): 93.75
Activity "PSP" mean duration time threshold (Q3 + 1.5IQR): 25.25
Activity "PoAS" mean duration time threshold (Q3 + 1.5IQR): 2.5
Activity "APB" mean duration time threshold (Q3 + 1.5IQR): 13.0
Activity "PIO" mean duration time threshold (Q3 + 1.5IQR): 2.5
Activity "R" mean duration time threshold (Q3 + 1.5IQR): 1.0


alt.VConcatChart(...)

In [12]:
log_data = log_data.merge(act_dur_thresholds, how='left', on='activity')
log_data

,activity,case_id,start timestamp,complete timestamp,resource,Weekday,Cost,"School holidays; 0 = no, 1 = yes",Approval; 1 = low risk to 4 = high risk,Type of loan; 0 = normal; 1 = special,_start timestamp,_complete timestamp,activity_duration_minute,activity_short,case_type,department,department_role,threshold
0,PIO,ID-21,2011-04-13 12:17:00,2011-04-13 12:17:00,010-24-26,Wednesday,"1,13 € per minute",0.0,1.0,1.0,2011-04-13 12:17:00,2011-04-13 12:17:00,0.0,PIO,special loan,010,010-24,2.500
1,PIO,ID-21,2011-04-14 10:53:00,2011-04-14 10:54:00,010-24-26,Thursday,"1,13 € per minute",0.0,1.0,1.0,2011-04-14 10:53:00,2011-04-14 10:54:00,1.0,PIO,special loan,010,010-24,2.500
2,PSP,ID-21,2011-04-14 11:00:00,2011-04-14 11:24:00,010-24-26,Thursday,"1,13 € per minute",0.0,1.0,1.0,2011-04-14 11:00:00,2011-04-14 11:24:00,24.0,PSP,special loan,010,010-24,25.250
3,PoAS,ID-21,2011-04-14 14:08:00,2011-04-14 14:09:00,010-24-26,Thursday,"1,13 € per minute",0.0,1.0,1.0,2011-04-14 14:08:00,2011-04-14 14:09:00,1.0,PoAS,special loan,010,010-24,2.500
4,AT,ID-21,2011-04-14 14:09:00,2011-04-14 15:04:00,010-24-26,Thursday,"1,13 € per minute",0.0,1.0,1.0,2011-04-14 14:09:00,2011-04-14 15:04:00,55.0,AT,special loan,010,010-24,93.750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,PoA,ID-1629,2011-03-31 08:39:00,2011-03-31 08:39:00,010-24-23,Thursday,"1,13 € per minute",0.0,1.0,1.0,2011-03-31 08:39:00,2011-03-31 08:39:00,0.0,PoA,special loan,010,010-24,75.000
278,GSoC,ID-1629,2011-03-31 08:40:00,2011-03-31 11:29:00,010-24-23,Thursday,"1,13 € per minute",0.0,1.0,1.0,2011-03-31 08:40:00,2011-03-31 11:29:00,169.0,GSoC,special loan,010,010-24,330.625
279,RB,ID-1629,2011-04-06 13:36:00,2011-04-06 13:36:00,010-24-23,Wednesday,"1,13 € per minute",0.0,1.0,1.0,2011-04-06 13:36:00,2011-04-06 13:36:00,0.0,RB,special loan,010,010-24,43.750
280,A,ID-1629,2011-04-06 13:37:00,2011-04-06 13:52:00,010-24-23,Wednesday,"1,13 € per minute",0.0,1.0,1.0,2011-04-06 13:37:00,2011-04-06 13:52:00,15.0,A,special loan,010,010-24,49.000


In [13]:
log_data = log_data[log_data['activity_duration_minute'] > log_data['threshold']]
log_data

,activity,case_id,start timestamp,complete timestamp,resource,Weekday,Cost,"School holidays; 0 = no, 1 = yes",Approval; 1 = low risk to 4 = high risk,Type of loan; 0 = normal; 1 = special,_start timestamp,_complete timestamp,activity_duration_minute,activity_short,case_type,department,department_role,threshold
6,PoAS,ID-21,2011-04-18 08:15:00,2011-04-18 08:33:00,010-24-26,Monday,"1,13 € per minute",1.0,1.0,1.0,2011-04-18 08:15:00,2011-04-18 08:33:00,18.0,PoAS,special loan,010,010-24,2.500
9,PoAS,ID-21,2011-05-06 12:11:00,2011-05-06 12:23:00,010-24-26,Friday,"1,13 € per minute",0.0,1.0,1.0,2011-05-06 12:11:00,2011-05-06 12:23:00,12.0,PoAS,special loan,010,010-24,2.500
36,PoAS,ID-208,2011-05-20 15:34:00,2011-05-20 15:52:00,010-24-26,Friday,"1,13 € per minute",0.0,2.0,1.0,2011-05-20 15:34:00,2011-05-20 15:52:00,18.0,PoAS,special loan,010,010-24,2.500
37,PoAS,ID-208,2011-05-20 15:56:00,2011-05-20 16:11:00,010-24-26,Friday,"1,13 € per minute",0.0,2.0,1.0,2011-05-20 15:56:00,2011-05-20 16:11:00,15.0,PoAS,special loan,010,010-24,2.500
70,PoA,ID-456,2011-05-04 09:13:00,2011-05-04 10:45:00,010-24-23,Wednesday,"1,13 € per minute",0.0,1.0,1.0,2011-05-04 09:13:00,2011-05-04 10:45:00,92.0,PoA,special loan,010,010-24,75.000
82,PoAS,ID-458,2011-03-03 12:06:00,2011-03-03 12:36:00,010-24-26,Thursday,"1,13 € per minute",0.0,1.0,1.0,2011-03-03 12:06:00,2011-03-03 12:36:00,30.0,PoAS,special loan,010,010-24,2.500
89,A,ID-458,2011-03-21 09:35:00,2011-03-21 14:21:00,010-24-02,Monday,"1,13 € per minute",0.0,1.0,1.0,2011-03-21 09:35:00,2011-03-21 14:21:00,286.0,A,special loan,010,010-24,49.000
105,PIO,ID-492,2011-04-15 14:58:00,2011-04-15 15:12:00,010-24-26,Friday,"1,13 € per minute",0.0,2.0,1.0,2011-04-15 14:58:00,2011-04-15 15:12:00,14.0,PIO,special loan,010,010-24,2.500
108,PSP,ID-492,2011-04-21 15:46:00,2011-04-21 16:30:00,010-24-26,Thursday,"1,13 € per minute",1.0,2.0,1.0,2011-04-21 15:46:00,2011-04-21 16:30:00,44.0,PSP,special loan,010,010-24,25.250
110,PoAS,ID-492,2011-04-21 16:31:00,2011-04-21 16:50:00,010-24-26,Thursday,"1,13 € per minute",1.0,2.0,1.0,2011-04-21 16:31:00,2011-04-21 16:50:00,19.0,PoAS,special loan,010,010-24,2.500


In [14]:
alt.layer(
    alt.Chart(
        log_data, 
        title='num. of performed activities with abnormally long completion time'
    ).mark_rect().encode(
        y=alt.Y('resource').title('resource ID'),
        x=alt.X('activity').title('activity'),
        color=alt.Color('count()').scale(scheme='lightgreyred').legend(None)
    ),
    alt.Chart(
        log_data
    ).mark_text(baseline='middle').encode(
        y=alt.Y('resource').title('resource ID'),
        x=alt.X('activity').title('activity'),
        text=alt.Text('count()')
    ),
).properties(width=320, height=80)

alt.LayerChart(...)